<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">François Rechenmann &amp; Thierry Parmentelat&nbsp;<img src="media/inria-25.png" style="display:inline"></span><br/>

# Recalage par recherche des RBS

Status:
  * code devrait être OK
  * mais reste à trouver les données
  * et à rédiger

In [ ]:
# la formule magique pour utiliser print() en python2 et python3
from __future__ import print_function
# pour que la division se comporte en python2 comme en python3
from __future__ import division

In [ ]:
def next_start_codon_after_rbs(arn, index, rbs):
    """
    recherche le premier start après une séquence RBS
    renvoie l'index du START, ou None si on n'en trouve plus
    """
    # on cherche le premier RBS
    index = arn.find(rbs, index)
    if index <= 1:
        return None
    index = arn.find(start_codon, index)
    if index <= 1:
        return None
    return index

In [ ]:
# avec à nouveau une longueur minimale de 300 par défaut
# et en utilisant le recalage RBS
def coding_regions_one_phase_rbs(adn, phase, minimal_length=300):
    # on initialise index à la phase; avec next_start_codon
    # et next_stop_codon on reste toujours sur la même phase
    index = phase
    # les résultats sont retournés sous la forme d'une liste 
    # de couples [start_gene, stop_gene]
    genes = []
    # on calcule stop1 qui désigne pour nous le stop "de gauche"
    # à ce stade, c'est le premier STOP à partir de la phase
    stop1 = next_stop_codon(adn, index)
    # s'il n'y a pas du tout de stop dans toute la séquence
    # on a terminé
    if not stop1:
        return genes
    # la boucle 
    while True:      # boucle principale
        # on cherche le premier STOP à partir de stop1
        # pour trouver le stop "de droite"
        stop2 = next_stop_codon(adn, stop1+3)
        # s'il n'y a plus de stop, on a fini
        if not stop2:
            return genes
        # mais il faut qu'il soit assez loin
        if stop2 - stop1 < minimal_length:
            # c'est trop court, on saute ce fragment
            stop1 = stop2
            continue
        # à ce stade on a trouvé un ORF, reste à trouver le bon start
        ##########
        # c'est ici la seule différence avec la version simple
        # de coding_regions_one_phase
        start = next_start_codon_after_rbs(adn, stop1, rbs)
        ##########
        # s'il n'y en a pas: c'est qu'on ne trouvera plus rien
        # et donc on a fini
        if not start:
            return genes
        # ici c'est un peu subtil
        # tout d'abord rien ne nous dit qu'il y a un codon START entre start1 et start2
        # et même s'il y en a un, rien de nous dit qu'on obtient un gène assez long
        # donc on reteste la longueur du gène trouvé et on ne garde que ceux 
        # qui sont assez longs - par rapport à minimal_length
        if stop2 - start < minimal_length:
            pass
        else:
            # cette fois on a trouvé un gène, on l'ajoute dans les résultats
            genes.append( [start, stop2] )
        # on peut passer à l'ORF suivant
        stop1 = stop2

XXX manque un exemple de RBS